<a href="https://colab.research.google.com/github/Tais005/ProjetoRegressaoLinearT3/blob/main/Projeto_Regress%C3%A3o_Linear.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.impute import SimpleImputer

from google.colab import drive
drive.mount('/content/drive')
url = '/content/drive/MyDrive/Colab/top_insta_influencers_data (1).csv'
data = pd.read_csv(url)

def transform_country(country):
    ifǰ isinstance(country, str):
        continent_mapping = {
            'South America': range(1, 10),
            'North America': range(20, 30),
            'Europe': range(40, 50),
            'Asia': range(60, 70),
            'Oceania': range(80, 90),
            'Africa': range(100, 110)
        }
        for continent, range_values in continent_mapping.items():
            if continent in country:
                return np.random.choice(range_values)
    return np.nan

data['country'] = data['country'].apply(transform_country)

numeric_columns = data.select_dtypes(include=['float64', 'int64']).columns
non_numeric_columns = data.select_dtypes(exclude=['float64', 'int64']).columns

imputer_numeric = SimpleImputer(strategy='mean')
data[numeric_columns] = imputer_numeric.fit_transform(data[numeric_columns])

imputer_non_numeric = SimpleImputer(strategy='most_frequent')
data[non_numeric_columns] = imputer_non_numeric.fit_transform(data[non_numeric_columns])

sns.pairplot(data[['followers', 'avg_likes', '60_day_eng_rate', 'total_likes']])
plt.show()

sns.scatterplot(x='followers', y='avg_likes', data=data)
plt.title('Relação entre Followers e Avg Likes')
plt.show()

X = data[['followers', 'avg_likes', '60_day_eng_rate', 'total_likes', 'country']]
y = data['influence_score']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

knn = KNeighborsRegressor()
knn.fit(X_train_scaled, y_train)

y_pred = knn.predict(X_test_scaled)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

print(f'MAE: {mae}')
print(f'MSE: {mse}')
print(f'RMSE: {rmse}')

cv_scores = cross_val_score(knn, X_train_scaled, y_train, cv=5, scoring='neg_mean_squared_error')
cv_rmse = np.sqrt(-cv_scores)
print(f'CV RMSE: {cv_rmse.mean()}')

param_grid = {'n_neighbors': [3, 5, 7, 9, 11], 'metric': ['euclidean', 'manhattan']}
grid_search = GridSearchCV(KNeighborsRegressor(), param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train_scaled, y_train)

best_knn = grid_search.best_estimator_
print(f'Best Parameters: {grid_search.best_params_}')
print(f'Best CV Score: {-grid_search.best_score_}')

sns.scatterplot(x='followers', y='avg_likes', data=data)
plt.title('Dispersão entre Followers e Avg Likes')
plt.show()

Mounted at /content/drive


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Colab/top_insta_influencers_data (1).csv'